<a href="https://colab.research.google.com/github/RenjieCui/OM_MIP/blob/main/UFL_and_ULS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import libraries
import numpy as np
import cvxpy as cp
import time
import matplotlib.pyplot as plt
import itertools
import sys
import random

!pip install cplex
!pip install docplex
from docplex.mp.model import Model

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 633.5/633.5 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docplex: filename=docplex-2.25.236-py3-none-any.whl size=671349 sha256=9aba932c66f017cdfeea7f384059ed986eb6a480d2ceaa0b3bafee2559117c97
  Stored in directory: /root/.cache/pip/wheels/3b/e5/00/0bf0173d67188fe73a13e3a61412b3f975f60205e3fab93a69
Successfully built docplex


In [ ]:
#big M model
ind = 0 # if (ind == 0), then we solve the original problem; if (ind == 1), then we solve its lp relaxation

def solve_UFL_1(m, n, f, c, ind):
  UFL_model = Model('UFL 1')

  x = []
  if (ind == 0):
    x = UFL_model.binary_var_list(n, name="x")
  if (ind == 1):
    x = UFL_model.continuous_var_list(n, name="x")

  y = UFL_model.continuous_var_matrix(m, n, name="y")

  #objective function
  obj_fn = sum(f[i]*x[i] for i in range(n)) + sum(sum(c[i, j]*y[i, j] for i in range(m)) for j in range(n))
  UFL_model.set_objective("min", obj_fn)

  #constraints
  for i in range(m):
    UFL_model.add_constraint(sum(y[i, j] for j in range(n)) == 1)

  for i in range(m):
    for j in range(n):
      UFL_model.add_constraint(y[i, j] >= 0)

  if (ind == 1):
    for j in range(n):
      UFL_model.add_constraint(x[j] >= 0)
      UFL_model.add_constraint(x[j] <= 1)

  for j in range(n):
    UFL_model.add_constraint(sum(y[i, j] for i in range(m)) <= m*x[j])

  sol = UFL_model.solve()

  if sol is None:
    print("Infeasible")

  obj_opt = UFL_model.solution.get_objective_value()
  return obj_opt

In [ ]:
#modified model
ind = 0 # if (ind == 0), then we solve the original problem; if (ind == 1), then we solve its lp relaxation

def solve_UFL_2(m, n, f, c, ind):
  UFL_model = Model('UFL 2')

  x = []
  if (ind == 0):
    x = UFL_model.binary_var_list(n, name="x")
  if (ind == 1):
    x = UFL_model.continuous_var_list(n, name="x")

  y = UFL_model.continuous_var_matrix(m, n, name="y")

  #objective function
  obj_fn = sum(f[i]*x[i] for i in range(n)) + sum(sum(c[i, j]*y[i, j] for i in range(m)) for j in range(n))
  UFL_model.set_objective("min", obj_fn)

  #old constraints
  for i in range(m):
    UFL_model.add_constraint(sum(y[i, j] for j in range(n)) == 1)

  for i in range(m):
    for j in range(n):
      UFL_model.add_constraint(y[i, j] >= 0)

  if (ind == 1):
    for j in range(n):
      UFL_model.add_constraint(x[j] >= 0)
      UFL_model.add_constraint(x[j] <= 1)

  #modified constraints
  for i in range(m):
    for j in range(n):
      UFL_model.add_constraint(y[i, j] <= x[j])

  sol = UFL_model.solve()

  if sol is None:
    print("Infeasible")

  obj_opt = UFL_model.solution.get_objective_value()
  return obj_opt

In [ ]:
# Models for UFL
# Parameters generation
n = 10 #number of depots
m = 10 #number of clients

average_quality_1 = 0
average_quality_2 = 0

# 100 experiments
num_exp = 100

for k in range(num_exp):
  f = np.zeros(n) #costs of using (opening) depots
  for i in range(n):
    f[i] = random.randint(1,100)

  c = np.zeros((m, n)) #transportion costs from clients to depots
  for i in range(m):
    for j in range(n):
      c[i, j] = random.randint(1,100)

  #print(f)
  #print(c)

  obj_opt_1 = solve_UFL_1(m, n, f, c, 0)
  obj_opt_relaxed_1 = solve_UFL_1(m, n, f, c, 1)

  obj_opt_2 = solve_UFL_2(m, n, f, c, 0)
  obj_opt_relaxed_2 = solve_UFL_2(m, n, f, c, 1)

  average_quality_1 += obj_opt_1 / obj_opt_relaxed_1 /num_exp
  average_quality_2 += obj_opt_2 / obj_opt_relaxed_2 /num_exp

print(average_quality_1, average_quality_2)

1.806885312260875 1.0012383410991477


In [ ]:
#ULS model 1
ind = 0 # if (ind == 0), then we solve the original problem; if (ind == 1), then we solve its lp relaxation

def solve_ULS_1(f, p, h, d, M, ind):
  ULS_model = Model('ULS 1')

  x = [] #x_t = 1 if production occurs in t, and x_t = 0 otherwise.
  if (ind == 0):
    x = ULS_model.binary_var_list(n, name="x")
  if (ind == 1):
    x = ULS_model.continuous_var_list(n, name="x")

  y = ULS_model.continuous_var_list(n, name="y") #the amount produced in period t
  s = ULS_model.continuous_var_list(n + 1, name="s") #stock at the end of period t - 1

  #objective function
  obj_fn = sum(f[t]*x[t] for t in range(n)) + sum(p[t]*y[t] for t in range(n)) + sum(h[t]*s[t + 1] for t in range(n))
  ULS_model.set_objective("min", obj_fn)

  #constraints
  for t in range(n):
    ULS_model.add_constraint(s[t] + y[t] == d[t] + s[t + 1])
    ULS_model.add_constraint(y[t] <= M*x[t])

  ULS_model.add_constraint(s[0] == 0)

  for t in range(n):
    ULS_model.add_constraint(y[t] >= 0)
    ULS_model.add_constraint(s[t + 1] >= 0)

  if (ind == 1):
    for i in range(n):
      ULS_model.add_constraint(x[i] >= 0)
      ULS_model.add_constraint(x[i] <= 1)

  sol = ULS_model.solve()

  if sol is None:
    print("Infeasible")

  obj_opt = ULS_model.solution.get_objective_value()

  #if (ind == 0):
  #  print("Model 2:", ULS_model.solution.get_values(x))
  return obj_opt

In [ ]:
#ULS model extended
ind = 0 # if (ind == 0), then we solve the original problem; if (ind == 1), then we solve its lp relaxation

def solve_ULS_2(f, p, h, d, ind):
  ULS_model = Model('ULS 2')

  x = [] #x_t = 1 if production occurs in t, and x_t = 0 otherwise.
  if (ind == 0):
    x = ULS_model.binary_var_list(n, name="x")
  if (ind == 1):
    x = ULS_model.continuous_var_list(n, name="x")

  #y = ULS_model.continuous_var_list(n, name="y") #the amount produced in period t
  #s = ULS_model.continuous_var_list(n + 1, name="s") #stock at the end of period t

  w = ULS_model.continuous_var_matrix(n, n, name="w") #the amount produced in period i to satisfy demand in period t

  c = np.zeros(n) #modified costs
  for t in range(n):
    c[t] = p[t] + sum(h[i] for i in range(t, n))

  #objective function
  K = sum(sum(d[i]*h[t] for i in range(t + 1)) for t in range(n))
  obj_fn = sum(f[t]*x[t] for t in range(n)) + sum(sum(c[i]*w[i, t] for t in range(i, n)) for i in range(n)) - K
  ULS_model.set_objective("min", obj_fn)

  #constraints
  for t in range(n):
    ULS_model.add_constraint(sum(w[i, t] for i in range(t + 1)) == d[t])
    for i in range(n):
      if (i <= t):
        ULS_model.add_constraint(w[i, t] >= 0)
        ULS_model.add_constraint(w[i, t] <= d[t]*x[i])

  if (ind == 1):
    for i in range(n):
      ULS_model.add_constraint(x[i] >= 0)
      ULS_model.add_constraint(x[i] <= 1)

  sol = ULS_model.solve()

  if sol is None:
    print("Infeasible")

  obj_opt = ULS_model.solution.get_objective_value()

  #if (ind == 0):
  #  print("Model 2:", ULS_model.solution.get_values(x))
  return obj_opt

In [ ]:
n = 5
p = [10, 15, 12, 8, 10]  # Variable cost for each period
h = [1, 1, 2, 1, 1]  # Storage cost for each period
f = [100, 100, 100, 100, 100]  # Fixed cost for each period
d = [20, 30, 25, 35, 30]  # Demand for each period

solve_ULS_1(f, p, h, d, M=200, ind=0)

1580.0

In [ ]:
solve_ULS_2(f, p, h, d, ind=0)

1580.0

In [ ]:
solve_ULS_1(f, p, h, d, M=200, ind=1)

1450.0

In [ ]:
solve_ULS_2(f, p, h, d, ind=1)

1580.0

In [ ]:
# Models for ULS
# Parameters generation
n = 10 #number of periods

average_quality_1 = 0
average_quality_2 = 0

# 100 experiments
num_exp = 100

for k in range(num_exp):
  f = np.zeros(n) #fixed costs of producing in period t
  p = np.zeros(n) #unit production cost in period t
  h = np.zeros(n) #unit storage cost in period t
  d = np.zeros(n) #demands in period t

  for t in range(n):
    f[t] = random.randint(1,100)
    p[t] = random.randint(1,100)
    h[t] = random.randint(1,100)
    d[t] = random.randint(1,100)

  M = 1000
  obj_opt_1 = solve_ULS_1(f, p, h, d, M, 0)
  obj_opt_relaxed_1 = solve_ULS_1(f, p, h, d, M, 1)

  obj_opt_2 = solve_ULS_2(f, p, h, d, 0)
  obj_opt_relaxed_2 = solve_ULS_2(f, p, h, d, 1)

  average_quality_1 += obj_opt_1 / obj_opt_relaxed_1 /num_exp
  average_quality_2 += obj_opt_2 / obj_opt_relaxed_2 /num_exp

print(average_quality_1, average_quality_2)

1.018682601394513 1.0000000000000007


In [ ]:
import gurobipy as gp
from gurobipy import GRB

def solve_lot_sizing_problem(n, p, h, f, d, lp_relaxation=False):
    try:
        # Create a Gurobi model
        model = gp.Model("Lot_Sizing")

        # Define decision variables
        x = model.addVars(n, vtype=GRB.BINARY if not lp_relaxation else GRB.CONTINUOUS, name="x")
        y = model.addVars(n, name="y")
        s = model.addVars(n+1, name="s")
        w = model.addVars(n, n, name="w")

        # Set the objective function
        model.setObjective(
            gp.quicksum(p[t] * y[t] + h[t] * s[t] + f[t] * x[t] for t in range(n)),
            GRB.MINIMIZE
        )

        # Inventory balance constraints
        model.addConstr(s[0] == 0)
        for t in range(1, n+1):
            model.addConstr(s[t-1] + y[t-1] == d[t-1] + s[t])

        # Demand fulfillment constraints
        for t in range(n):
            model.addConstr(gp.quicksum(w[i, t] for i in range(t+1)) == d[t])

        # w and x relationship constraints
        for t in range(n):
            for i in range(t+1):
                model.addConstr(w[i, t] <= d[t] * x[t])
                model.addConstr(w[i, t] >= 0)

        # Define y_i as the sum of w_i,t for t in [i, n]
        for i in range(n):
            model.addConstr(y[i] == gp.quicksum(w[i, t] for t in range(i, n)))

        # Solve the model
        if lp_relaxation:
            model.Params.Method = 1  # Use dual simplex for LP relaxation

        model.Params.OutputFlag = 0  # Suppress Gurobi output
        model.optimize()

        # Check and return the results
        if model.status == GRB.OPTIMAL:
            order_quantities = [x[t].x for t in range(n)]
            objective_value = model.objVal
            return {
                "status": "Optimal",
                "order_quantities": order_quantities,
                "objective_value": objective_value
            }
        else:
            return {"status": "No optimal solution found"}

    except gp.GurobiError as e:
        print(f"Gurobi Error: {e}")
        return {"status": "Error"}

# Example usage
n = 5
p = [10, 12, 14, 11, 9]
h = [1, 2, 2, 1, 3]
f = [100, 120, 150, 180, 200]
d = [50, 60, 40, 70, 30]

# Solve using MILP
result_milp = solve_lot_sizing_problem(n, p, h, f, d, lp_relaxation=False)
if result_milp["status"] == "Optimal":
    print("MILP Solution Found:")
    print("Order Quantities (x):", result_milp["order_quantities"])
    print("Objective Value:", result_milp["objective_value"])
else:
    print("No MILP optimal solution found.")

# Solve using LP relaxation
result_lp_relax = solve_lot_sizing_problem(n, p, h, f, d, lp_relaxation=True)
if result_lp_relax["status"] == "Optimal":
    print("\nLP Relaxation Solution Found:")
    print("Order Quantities (x):", result_lp_relax["order_quantities"])
    print("Objective Value:", result_lp_relax["objective_value"])
else:
    print("No LP relaxation optimal solution found.")


In [ ]:
# Uncapacitated Lot-Sizing under Extended Formulation
# Parameters
# p: unit production cost in period t
# h: unit storage cost in period t
# f: fixed cost in period t
# d: demand in period t


def solve_ULS_1(p, h, f, d, lp_relaxation=False):
    try:
        n = len(p)  # Planning horizon

        # Create the model
        model = gp.Model("Lot_Sizing_Extended")

        # Define decision variables
        w = model.addVars(n, n, name="w")
        s = model.addVars(n+1, name="s")
        y = model.addVars(n, name="y")
        x = model.addVars(n, vtype=GRB.BINARY, name="x")

        # Set the objective function
        model.setObjective(
            gp.quicksum(p[t] * y[t] for t in range(n)) +
            gp.quicksum(h[t] * s[t + 1] for t in range(n)) +
            gp.quicksum(f[t] * x[t] for t in range(n)),
            GRB.MINIMIZE
        )

        # Inventory balance constraints
        model.addConstr(s[0] == 0) # Zero inital inventory constraint
        for t in range(n):
            model.addConstr(s[t] + y[t] == d[t] + s[t + 1])

        for t in range(n):
            model.addConstr(gp.quicksum(w[i, t] for i in range(t + 1)) == d[t])

        # Upper-bound constraints
        for t in range(n):
            # model.addConstr(gp.quicksum(w[i, t] for i in range(t + 1)) == d[t])
            for i in range(n):
                if (i <= t):
                    model.addConstr(w[i, t] >= 0)
                    model.addConstr(w[i, t] <= d[t] * x[t])

        # Production in future periods cannot be used to satisfy demand in past periods
        # for i in range(n):
        #     for t in range(i, n):
        #         model.addConstr(w[i, t] >= 0)

        for i in range(n):
            model.addConstr(y[i] == gp.quicksum(w[i, t] for t in range(i, n)))
                # model.addConstr(y[i] == gp.quicksum(w[i, t] for t in range(i, n)))

        # LP relaxation
        if lp_relaxation:
            for t in range(n):
                x[t].vtype = GRB.CONTINUOUS
                model.addConstr(x[t] >= 0)
                model.addConstr(x[t] <= 1)

        # Solve the model
        model.Params.OutputFlag = 0  # Suppress Gurobi output
        # model.setParam("FeasibilityTol", 10e-8)
        model.setParam("OptimalityTol", 1e-9)
        model.optimize()


        # Check for optimality
        if model.status == GRB.OPTIMAL:
            solution = {
                "x": [x[t].x for t in range(n)],
                "y": [y[t].x for t in range(n)],
                "s": [s[t].x for t in range(n)],
                "total_cost": model.objVal
            }
            return solution
        else:
            return None
    except gp.GurobiError as e:
        print(f"Gurobi Error: {e}")
        return None